walmart 주단위 데이터 139주로 향후 4주 예측하기

In [202]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime  
columns=['date','sales']
txs=pd.read_table('./lstmData/lstmPrac12.csv', sep=',',header=None,names=columns )
tf.reset_default_graph()

In [203]:
txs

,date,sales
0,2010-02-05,24924.50
1,2010-02-12,46039.49
2,2010-02-19,41595.55
3,2010-02-26,19403.54
4,2010-03-05,21827.90
5,2010-03-12,21043.39
6,2010-03-19,22136.64
7,2010-03-26,26229.21
8,2010-04-02,57258.43
9,2010-04-09,42960.91


In [204]:
def noOutlierSales(sales):
    mean=np.mean(sales)
    std=np.std(sales)
    for i in range(len(sales)):
        if (sales[i]<mean-2*std or sales[i]>mean+2*std):
             sales[i]=int(mean)
    return sales
def logSales(sales):
    for i in range(len(sales)):
        if sales[i] is 0:
            sales[i]=1
    return np.log(sales)
def sqrtSales(sales):
    return np.sqrt(sales)


def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier


In [205]:
def LSTM(txs, forecastDay, features):
    tf.set_random_seed(7)
    #Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['date'].map(year)
    txs['month'] = txs['date'].map(month)
    txs['weekNumber'] = txs['date'].map(weekNumber)
    txs['dayOfWeek'] = txs['date'].map(dayOfWeek)

    #Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['date'].map(season)
    txs['dayOfWeek01'] = txs['date'].map(day_of_week01)

    #Backup originalSales
    originalSales = list(txs['sales'])
    sales = list(txs['sales'])

    if features is 'DayOfWeek_WeekNumber_Month_Season' :
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']),list(txs['month']),list(txs['season']) , sales]
    elif features is'DayOfWeek01_WeekNumber_Month_Season' :
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']),list(txs['month']),list(txs['season']) , sales]

    elif features is 'WeekNumber_Month_Season_Year' :
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']),list(txs['year']), sales]

    xy = np.array(tempxy).transpose().astype(np.float)

    #Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    #TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001

    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay+1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))

        dataX.append(_x)
        dataY.append(_y)
  
    train_size = int(len(dataY)-2*forecastDay)
    train_size = int(len(dataY)*0.7)    
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    
    denormalizedTestY=originalSales[train_size+seq_length:]
#     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])
    
    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])
    
    count=0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        while(1):
            count=count+1
            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(count, step_loss))
            if(step_loss<1):
                break

        # Test step
        # test_predict= sess.run(Y_pred, feed_dict={X: testX}
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}),originalXY)
        realSale= minMaxDeNormalizer(testY[-1],originalXY)
        # Plot predictions
#         plt.plot(denormalizedTestY_feed) #실제 sales 파란색
        plt.plot(realSale)      #실제 sales 파란색
        plt.plot(test_predict[-1]) #예측 sales 주황색
               
        plt.xlabel("Time Period")
        plt.ylabel("Stock Price")
        plt.show()
        
    return test_predict[-1]




In [ ]:
answer=LSTM(txs,7,'WeekNumber_Month_Season_Year' )

[[  8.68174327e-05   3.47269731e-05   0.00000000e+00   3.49006079e-02
    4.32776220e-01]
 [  1.04180919e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
    7.99406065e-01]
 [  1.21544406e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
    7.22243772e-01]
 [  1.38907892e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
    3.36913105e-01]
 [  1.56271379e-04   5.20904596e-05   1.73634865e-05   3.49006079e-02
    3.79008448e-01]
 [  1.73634865e-04   5.20904596e-05   1.73634865e-05   3.49006079e-02
    3.65386619e-01]
 [  1.90998352e-04   5.20904596e-05   1.73634865e-05   3.49006079e-02
    3.84369251e-01]
 [  2.08361838e-04   5.20904596e-05   1.73634865e-05   3.49006079e-02
    4.55430535e-01]
 [  2.25725325e-04   6.94539461e-05   1.73634865e-05   3.49006079e-02
    9.94205978e-01]
 [  2.43088811e-04   6.94539461e-05   1.73634865e-05   3.49006079e-02
    7.45951182e-01]] -> [ 0.30554458  0.28033957  0.28745443  0.30236671  0.3286342   0.25651148
  0.27053059]
[[  1.041

[[  5.90358542e-04   1.38907892e-04   3.47269731e-05   3.49006079e-02
    2.74236649e-01]
 [  6.07722029e-04   1.56271379e-04   5.20904596e-05   3.49006079e-02
    2.82013928e-01]
 [  6.25085515e-04   1.56271379e-04   5.20904596e-05   3.49006079e-02
    3.15924123e-01]
 [  6.42449002e-04   1.56271379e-04   5.20904596e-05   3.49006079e-02
    3.36056912e-01]
 [  6.59812488e-04   1.56271379e-04   5.20904596e-05   3.49006079e-02
    3.14670132e-01]
 [  6.77175975e-04   1.73634865e-04   5.20904596e-05   3.49006079e-02
    3.48905197e-01]
 [  6.94539461e-04   1.73634865e-04   5.20904596e-05   3.49006079e-02
    4.06097744e-01]
 [  7.11902948e-04   1.73634865e-04   5.20904596e-05   3.49006079e-02
    4.68438043e-01]
 [  7.29266434e-04   1.73634865e-04   5.20904596e-05   3.49006079e-02
    4.43516231e-01]
 [  7.46629921e-04   1.73634865e-04   5.20904596e-05   3.49006079e-02
    6.70941268e-01]] -> [ 0.59450633  0.33944557  0.33950547  0.32678585  0.39098335  0.54690902
  0.77984384]
[[  6.077

trainX: [[[  8.68174327e-05   3.47269731e-05   0.00000000e+00   3.49006079e-02
     4.32776220e-01]
  [  1.04180919e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
     7.99406065e-01]
  [  1.21544406e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
     7.22243772e-01]
  ..., 
  [  2.08361838e-04   5.20904596e-05   1.73634865e-05   3.49006079e-02
     4.55430535e-01]
  [  2.25725325e-04   6.94539461e-05   1.73634865e-05   3.49006079e-02
     9.94205978e-01]
  [  2.43088811e-04   6.94539461e-05   1.73634865e-05   3.49006079e-02
     7.45951182e-01]]

 [[  1.04180919e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
     7.99406065e-01]
  [  1.21544406e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
     7.22243772e-01]
  [  1.38907892e-04   3.47269731e-05   0.00000000e+00   3.49006079e-02
     3.36913105e-01]
  ..., 
  [  2.25725325e-04   6.94539461e-05   1.73634865e-05   3.49006079e-02
     9.94205978e-01]
  [  2.43088811e-04   6.94539461e-05   1.73634865e-05 

[step: 43] loss: 17.00921058654785
[step: 44] loss: 16.85532569885254
[step: 45] loss: 16.719663619995117
[step: 46] loss: 16.628978729248047
[step: 47] loss: 16.590730667114258
[step: 48] loss: 16.59581756591797
[step: 49] loss: 16.625457763671875
[step: 50] loss: 16.65873146057129
[step: 51] loss: 16.678295135498047
[step: 52] loss: 16.67374610900879
[step: 53] loss: 16.64259147644043
[step: 54] loss: 16.589332580566406
[step: 55] loss: 16.523361206054688
[step: 56] loss: 16.45618438720703
[step: 57] loss: 16.39847755432129
[step: 58] loss: 16.357582092285156
[step: 59] loss: 16.335926055908203
[step: 60] loss: 16.330806732177734
[step: 61] loss: 16.335590362548828
[step: 62] loss: 16.342084884643555
[step: 63] loss: 16.34317398071289
[step: 64] loss: 16.33486557006836
[step: 65] loss: 16.31699562072754
[step: 66] loss: 16.292564392089844
[step: 67] loss: 16.266094207763672
[step: 68] loss: 16.241832733154297
[step: 69] loss: 16.22243881225586
[step: 70] loss: 16.2085018157959
[step:

[step: 272] loss: 13.472288131713867
[step: 273] loss: 13.464508056640625
[step: 274] loss: 13.456694602966309
[step: 275] loss: 13.448858261108398
[step: 276] loss: 13.44098949432373
[step: 277] loss: 13.433095932006836
[step: 278] loss: 13.425167083740234
[step: 279] loss: 13.417207717895508
[step: 280] loss: 13.409215927124023
[step: 281] loss: 13.401185989379883
[step: 282] loss: 13.393121719360352
[step: 283] loss: 13.385009765625
[step: 284] loss: 13.376856803894043
[step: 285] loss: 13.368657112121582
[step: 286] loss: 13.360404968261719
[step: 287] loss: 13.35209846496582
[step: 288] loss: 13.343729972839355
[step: 289] loss: 13.335306167602539
[step: 290] loss: 13.326813697814941
[step: 291] loss: 13.318248748779297
[step: 292] loss: 13.309621810913086
[step: 293] loss: 13.300919532775879
[step: 294] loss: 13.292163848876953
[step: 295] loss: 13.283377647399902
[step: 296] loss: 13.274641036987305
[step: 297] loss: 13.266167640686035
[step: 298] loss: 13.258394241333008
[step:

[step: 503] loss: 11.740560531616211
[step: 504] loss: 11.656733512878418
[step: 505] loss: 11.768970489501953
[step: 506] loss: 11.90071964263916
[step: 507] loss: 11.740867614746094
[step: 508] loss: 11.63220500946045
[step: 509] loss: 11.67884635925293
[step: 510] loss: 11.739508628845215
[step: 511] loss: 11.715609550476074
[step: 512] loss: 11.617391586303711
[step: 513] loss: 11.641273498535156
[step: 514] loss: 11.705401420593262
[step: 515] loss: 11.643777847290039
[step: 516] loss: 11.591592788696289
[step: 517] loss: 11.621219635009766
[step: 518] loss: 11.639966011047363
[step: 519] loss: 11.610334396362305
[step: 520] loss: 11.571792602539062
[step: 521] loss: 11.58854866027832
[step: 522] loss: 11.608646392822266
[step: 523] loss: 11.57687759399414
[step: 524] loss: 11.551305770874023
[step: 525] loss: 11.563413619995117
[step: 526] loss: 11.568278312683105
[step: 527] loss: 11.550103187561035
[step: 528] loss: 11.529928207397461
[step: 529] loss: 11.534467697143555
[step:

In [ ]:
len(answer)

In [ ]:
predict=answer[0]

In [ ]:
real=answer[1]

In [ ]:
predict

In [ ]:
real

In [ ]:
rmse(predict,real)

In [ ]:
denormalizedTestPredictY=[item for sublist in predict for item in sublist]

In [ ]:
denormalizedTestPredictY

In [ ]:
denormalizedTestY=originalSales[train_size+seq_length:]

In [ ]:
# sales=noOutlierSales(sales)
tempxy=[list(txs['season']),list(txs['year']),list(txs['month']),list(txs['week_number']),sales]
# tempxy=[list(txs['season']),list(txs['day_of_week01']),list(txs['week_number']),sales]
xy=np.array(tempxy).transpose().astype(np.float)
originalxy=np.array(tempxy).transpose().astype(np.float)
xy=minMaxNormalizer(xy)

#data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
data_dim=len(tempxy)
#data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
seq_length=5
#output_dim(=forecastDays)만큼의 다음날 y_data를 예측

output_dim=forecastDay
#hidden_dim은 정말 임의로 설정
hidden_dim=100
#learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
learning_rate=0.01
#iterations는 반복 횟수
iterations=1000
x=xy
y=xy[:,[-1]]

#build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
dataX=[]
dataY=[]
for i in range(0, len(y)-seq_length - forecastDay):
    _x=x[i:i+seq_length]
    _y = y[i + seq_length:i + seq_length + forecastDay]
    _y = np.reshape(_y, (forecastDay))
#     print(_x,"->",_y)
    dataX.append(_x)
    dataY.append(_y)
    train_size = int(len(dataY) * 0.7)
    
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])
X=tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y=tf.placeholder(tf.float32, [None, forecastDay])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn= None) 
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)
denormalizedTestY=originalSales[train_size+seq_length:]
# denormalizedTestY_original=sales[train_size+seq_length:]
denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))
with tf.Session() as sess:
    #초기화
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    # Test step
    test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}),originalxy)

    # Plot predictions
    plt.plot(denormalizedTestY_feed) #실제 sales 파란색
    plt.plot(test_predict)           #예측 sales 주황색
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()

In [ ]:
print(i for i in list(test_predict[-1]    ))

In [5]:
list(test_predict[  )

[17721.039]

In [6]:
denormalizedTestPredictY=[item for sublist in test_predict for item in sublist]

In [7]:
rootMeanSquaredError(denormalizedTestY,denormalizedTestPredictY)

IndexError: list index out of range

In [ ]:
len(denormalizedTestPredictY)

In [ ]:
denormalizedTestY